In [345]:
from math import log
from line_profiler import LineProfiler

In [346]:
%load_ext line_profiler

## Replace the divide instruction by multiply and shift, or adds and shifts

Source:  
https://en.m.wikipedia.org/wiki/Division_algorithm#Division_by_a_constant    
https://stackoverflow.com/questions/5558492/divide-by-10-using-bit-shifts  
OEIS

The OEIS sequences [A346495](https://oeis.org/A346495) and [A346496](https://oeis.org/A346496) are used for replacing division with multiplication via an approximation of the inverse of the divisor, if the computer supports multiplying two 32-bit numbers for a 64-bit result.  
This is usually much faster than a division instruction because integer division is a very slow operation on most computers.

Smallest a(n) so that division by n can be performed by $$\left\lfloor\frac{x}{n}\right\rfloor = \frac{\lfloor(x*a(n)\rfloor}{2^{b(n)}}$$ for all $0 <= x < 2^{32}$. 

In [1]:
def power_of_two(n):
    return n == (1<<(n.bit_length()-1))

In [2]:
def a(n):
    Result = 1
    if not power_of_two(n):
        n_c = ((1<<32) - ((1<<32) % n)) - 1
        b   = 1
        while (1<<b) <= (n_c * ((n - 1) - (((1<<b) - 1) % n))):
            b += 1
        Result = ((1<<b) + (n - 1) - (((1<<b) - 1) % n)) // n
    return Result

In [4]:
def b(n):
    Result = None
    if power_of_two(n):
        Result = n.bit_length() - 1
    else:
        n_c = ((1<<32) - ((1<<32) % n)) - 1
        b   = 1
        while (1<<b) <= (n_c * ((n - 1) - (((1<<b) - 1) % n))):
            b += 1
        Result = b
    return Result

In [3]:
print([a(n) for n in range(1, 26)])

[1, 1, 2863311531, 1, 3435973837, 2863311531, 4908534053, 1, 954437177, 3435973837, 3123612579, 2863311531, 1321528399, 4908534053, 2290649225, 1, 4042322161, 954437177, 7233629131, 3435973837, 6544712071, 3123612579, 2987803337, 2863311531, 1374389535]


In [15]:
print([b(n) for n in range(1, 26)])

[0, 1, 33, 2, 34, 34, 35, 3, 33, 35, 35, 35, 34, 36, 35, 4, 36, 34, 37, 36, 37, 36, 36, 36, 35]


Here we compute both a and b at the same time.

In [20]:
LIMIT     = 32
BIT_LIMIT = 1<<LIMIT
def seq(n):
    A = 1
    B = n.bit_length() - 1
    j = n - 1
    if not (n == (1<<B)):
        k = (BIT_LIMIT - (BIT_LIMIT % n)) - 1
        B   = 1
        B2  = 1<<B
        while B2 <= (k * (j - ((B2 - 1) % n))):
            B += 1
            B2 = B2<<1
        A = (B2 + j - ((B2 - 1) % n)) // n
    return (A, B)

In [19]:
print([seq(n) for n in range(1, 26)])

[(1, 0), (1, 1), (2863311531, 33), (1, 2), (3435973837, 34), (2863311531, 34), (4908534053, 35), (1, 3), (954437177, 33), (3435973837, 35), (3123612579, 35), (2863311531, 35), (1321528399, 34), (4908534053, 36), (2290649225, 35), (1, 4), (4042322161, 36), (954437177, 34), (7233629131, 37), (3435973837, 36), (6544712071, 37), (3123612579, 36), (2987803337, 36), (2863311531, 36), (1374389535, 35)]


### For our usecase, we would like to operate on numbers much larger than 32 or even 64 bits.  
### Luckily there is another way to compute these "magic numbers".

Source:  
Hacker's delight page 218 (chapter 10–15 Simple Code in Python)

In [23]:
def magicgu_slow(nmax, d):
    dminus1      = d - 1
    nc           = (nmax // d) * dminus1
    search_range = 1 + (2 * nmax.bit_length())
    for p in range(0, search_range):
        q         = 1<<p
        qminus1   = q - 1
        remainder = (qminus1 % d)
        if q > (nc * (dminus1 - remainder)):
            m = ((q + dminus1) - remainder) // d
            return (m, p)
    raise Exception('Couldn\'t find a solution, something is wrong.')

Optimization:  
Switching from a linear search to a binary search reduces the runtime for inputs `m, p = magicgu(1<<1_000_000, 10**399)` from 40 minutes to 0.08 seconds, taking just 24 iterations to converge on the correct value.

In [24]:
def magicgu(nmax, d):
    """
    Given the maximum value of the dividend nmax and the divisor d, returns a pair of integers:
        the magic number m and a shift amount p.
    To divide a dividend x by d, one multiplies x by m and then shifts the (full length) product right p bits.
    This works because m / p will be very close to 1 / d, and p will be a power of two, making the divide a simple bit shift.
    """
    dminus1 = d - 1
    nc      = (nmax // d) * dminus1
    lowest  = 0
    highest = 1 + (2 * nmax.bit_length())
    while lowest < highest:
        p         = (highest + lowest) // 2
        q         = 1<<p
        qminus1   = q - 1
        remainder = (qminus1 % d)
        if q > (nc * (dminus1 - remainder)):
            highest = p
        else:
            lowest  = p + 1
    p         = (highest + lowest) // 2
    q         = 1<<p
    qminus1   = q - 1
    remainder = (qminus1 % d)
    m         = ((q + dminus1) - remainder) // d
    return (m, p)

#### Example 1

In [42]:
nmax = 1<<1_000_000
d    = 10**399

In [44]:
m, p = magicgu(nmax, d)

In [45]:
x = 1<<999_900

In [46]:
assert x < nmax

In [47]:
%%timeit
(x * m) >> p

4.37 ms ± 149 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [48]:
%%timeit
x // d

3.02 ms ± 186 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### Example 2

In [36]:
nmax = 10**399
d    = 10

In [37]:
m, p = magicgu(nmax, d); m, p

(292957247209924852137075224890370171032692958681326960826984919130001827539839214697757948051204538105646676347303848625162902618953473698102408269423433879425259310757283275210972425883538122452847336119254634643499505917091533837488298906776482947295647220679792987618467923311462718229513113755678107850012507556042175979861246759917806100226846569645613245309343003296469005736181778637822676173,
 1327)

In [38]:
# An example number we've come across in the algorithm we're trying to optimize.
x = 675498068352129731964884062763533329144415921218630285019127698639680940044270154133162810213979906631608629547659877222303938885553947675194806514177810321278967470122934933377995913130368225534489724572693654861728928532250033597205506631158056720564410536312378792296760105033917922985829102352323964365789020903947560236392997251191841134769795404868939332473278193403660671793915429281166670196

In [39]:
assert x < nmax

In [40]:
%%timeit
(x * m) >> p

3.32 µs ± 105 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [41]:
%%timeit
x // 10

268 ns ± 11.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


#### Result

This works, but sadly it's not really faster than the usual division, even when the number we're dividing is close to the maximum (which makes the difference between the mult and shift version and the division version the smallest).

### If the divisor and the maximum size are fixed (which they are in our case), we can go another approach which even gets rid of the multiply instruction and instead does a bunch of adds and shifts  
### Sadly this implementation again only works for 32 bit integers

Source:  
Hackers Delight (see above)

In [50]:
def divu10(n):
    q  = (n >> 1) + (n >> 2)
    q += (q >> 4)
    q += (q >> 8)
    q += (q >> 16)
    q  = (q >> 3)
    r  = n - (((q << 2) + q) << 1)
    return q + (r > 9), r
divu10(21)

(2, 11)

In [51]:
def divu100(n):
    q = (n >> 1) + (n >> 3) + (n >> 6) - (n >> 10) + (n >> 12) + (n >> 13) - (n >> 16)
    q = q + (q >> 20)
    q = q >> 6
    r = n - q*100
    return q + ((r + 28) >> 7)
    # // return q + (r > 99)
divu100(240)

2

In [52]:
def divu1000(n):
    t = (n >> 7) + (n >> 8) + (n >> 12)
    q = (n >> 1) + t + (n >> 15) + (t >> 11) + (t >> 14)
    q = q >> 9
    r = n - q*1000
    return q + ((r + 24) >> 10)
    # // return q + (r > 999)
divu1000(3600)

3

This would need to be adapter to larger numbers, and could then be faster, but definitely wont be in python. Maybe try this when we have an implementation in C.

## Speed up modulo by using remainder instead of entire number

Testing if we can use the remainder of a division with a large multiple of 10 to speed up the computation of modulo 10.

NOTE: There seems to be something fishy going on with the last digit, depending on which exponent we choose. All exponents of two ending in 0 seem to be ending in 6 once calculated. All with 1 end with 2, all with 2 end with 4 and so on. Check this out later!

In [11]:
N = 1<<1001

In [12]:
one = N % 10**1

In [13]:
two = N % 10**2

In [14]:
three = N % 10**3

In [15]:
four = N % 10**4

In [16]:
five = N % 10**5

In [17]:
six = N % 10**6

In [18]:
seven = N % 10**7

In [19]:
eight = N % 10**8

In [20]:
one, two, three, four, five, six, seven, eight

(2, 52, 752, 8752, 38752, 138752, 6138752, 36138752)

In [122]:
N

42860344287450692837937001962400072422456192468221344297750015534814042044997444899727935152627834325103786916702125873007485811427692561743938310298794299215738271099296923941684298420249484567511816728612185899934327765069595070236662175784308251658284785910746168670641719326610497547348822672277504

In [123]:
N % 10**399

42860344287450692837937001962400072422456192468221344297750015534814042044997444899727935152627834325103786916702125873007485811427692561743938310298794299215738271099296923941684298420249484567511816728612185899934327765069595070236662175784308251658284785910746168670641719326610497547348822672277504

It seems like any time we do a `n mod 10**k`, we're really grabbing about the last k decimal digits of the number n. So if we then do a mod 10 afterwards on that value, we will get the result of n mod 10, without having to do the whole computation again.  
With this realisation it's obvious that we can just take the modulo of the remainder instead of the entire number, saving massively on computation.

In [209]:
profiler = LineProfiler()
@profiler
def count_sixes(N, D, Cache):
    SixCounter    = 0
    SixCounterMax = 0
    j = 0
    while N > 0:
        # TODO: use continued division for this as well? Or can we just use the remainder instead of the number?
        Digit = N % 10
        if Digit == 6:
            SixCounter   += 1
            SixCounterMax = max(SixCounterMax, SixCounter)
        else:
            SixCounter = 0
        try:
            Q, R = Cache[j]
            R    = (R << 1) | (N & 1)
            if R >= D:
                R = R - D
                N = (Q << 1) | 1
            else:
                N = (Q << 1)
        except KeyError:
            R = N  % D
            N = N // D
        Cache[j] = N, R
        j += 1
    return SixCounterMax

In [213]:
# This is the new version
profiler = LineProfiler()
@profiler
def count_sixes(N, D, Cache):
    SixCounter    = 0
    SixCounterMax = 0
    j = 0
    while N > 0:
        try:
            Q, R = Cache[j]
            R    = (R << 1) | (N & 1)
            if R >= D:
                R = R - D
                N = (Q << 1) | 1
            else:
                N = (Q << 1)
        except KeyError:
            R = N  % D
            N = N // D
        Cache[j] = N, R
        j += 1
        
        Digit = R % 10
        if Digit == 6:
            SixCounter   += 1
            SixCounterMax = max(SixCounterMax, SixCounter)
        else:
            SixCounter = 0
        
    return SixCounterMax

In [214]:
%%time
i        = 100_000
Number   = 1<<(i - 1)
print('Building Cache...')
Divisor = 10**399
Cache   = {}
count_sixes(Number, Divisor, Cache)
Number = Number << 1
print('Done!')

Building Cache...
Done!
CPU times: total: 31.2 ms
Wall time: 18 ms


In [211]:
%%time
output = []
for _ in range(1000):
    output.append(count_sixes(Number, Divisor, Cache))
    i      += 1
    Number  = Number<<1
print(output)

[2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 3, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 3, 1, 1, 1, 1, 1, 1, 2, 2, 3, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 6, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 3, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 3, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 3, 1, 1, 3, 2, 1, 2, 1, 1, 3, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 2, 1, 2, 3, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 3, 2, 1, 2, 1, 2, 2, 1, 3, 1, 1, 2, 2, 1, 2, 1, 3, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 3, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 3, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 3, 1, 1, 1, 3, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 4, 1, 1, 3, 1, 2, 1, 1, 1, 1, 3, 2, 1, 2, 2, 

In [215]:
%%time
output = []
for _ in range(1000):
    output.append(count_sixes(Number, Divisor, Cache))
    i      += 1
    Number  = Number<<1
print(output)

[2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 3, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 3, 1, 1, 1, 1, 1, 1, 2, 2, 3, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 6, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 3, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 3, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 3, 1, 1, 3, 2, 1, 2, 1, 1, 3, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 2, 1, 2, 3, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 3, 2, 1, 2, 1, 2, 2, 1, 3, 1, 1, 2, 2, 1, 2, 1, 3, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 3, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 3, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 3, 1, 1, 1, 3, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 4, 1, 1, 3, 1, 2, 1, 1, 1, 1, 3, 2, 1, 2, 2, 

In [207]:
[2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 3, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 3, 1, 1, 1, 1, 1, 1, 2, 2, 3, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 6, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 3, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 3, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 3, 1, 1, 3, 2, 1, 2, 1, 1, 3, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 2, 1, 2, 3, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 3, 2, 1, 2, 1, 2, 2, 1, 3, 1, 1, 2, 2, 1, 2, 1, 3, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 3, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 3, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 3, 1, 1, 1, 3, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 4, 1, 1, 3, 1, 2, 1, 1, 1, 1, 3, 2, 1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 3, 2, 3, 4, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 3, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 3, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 3, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 3, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 3, 2, 3, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 3, 1, 1, 2, 1, 1, 1, 2, 4, 1, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 3, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 3, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 3, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 3, 1, 2, 1, 1, 1, 4, 3, 1, 1, 2, 2, 1, 3, 1, 1, 1, 1, 2, 3, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 3, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 3, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 3, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 4, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 3, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 3, 2, 3, 1, 1, 1, 1, 2, 3, 1, 3, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 3, 1, 2, 2, 2, 3, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 3, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 3, 1, 1, 1, 2, 3, 1, 1, 1, 3, 3, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 3, 1, 2, 2, 2, 3, 1, 3, 1, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 3, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 3, 1, 2, 3, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2] == [2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 3, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 2, 3, 1, 1, 1, 1, 1, 1, 2, 2, 3, 2, 3, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 6, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 3, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 3, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 2, 3, 1, 1, 3, 2, 1, 2, 1, 1, 3, 2, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 2, 2, 1, 2, 3, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 3, 2, 1, 2, 1, 2, 2, 1, 3, 1, 1, 2, 2, 1, 2, 1, 3, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 3, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 3, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 3, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 2, 3, 1, 1, 1, 3, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 4, 1, 1, 3, 1, 2, 1, 1, 1, 1, 3, 2, 1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 3, 2, 3, 4, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 3, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 3, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 3, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 3, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 3, 2, 3, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 3, 1, 3, 1, 1, 2, 1, 1, 1, 2, 4, 1, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 3, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 3, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 2, 1, 3, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 3, 1, 2, 1, 1, 1, 4, 3, 1, 1, 2, 2, 1, 3, 1, 1, 1, 1, 2, 3, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1, 2, 2, 3, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 1, 1, 2, 1, 1, 1, 2, 1, 3, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 3, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 4, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 3, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 3, 2, 3, 1, 1, 1, 1, 2, 3, 1, 3, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 2, 3, 1, 2, 2, 2, 3, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 1, 3, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 3, 1, 1, 1, 2, 3, 1, 1, 1, 3, 3, 2, 1, 2, 2, 2, 2, 1, 2, 2, 1, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 3, 1, 2, 2, 2, 3, 1, 3, 1, 3, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 3, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 3, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 3, 1, 2, 3, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 2]

True

In [212]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 0.7823 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_16104\356962432.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(N, D, Cache):
     4      1001       2489.0      2.5      0.0      SixCounter    = 0
     5      1001       2437.0      2.4      0.0      SixCounterMax = 0
     6      1001       2445.0      2.4      0.0      j = 0
     7     77342     190859.0      2.5      2.4      while N > 0:
     8                                                   # TODO: use continued division for this as well? Or can we just use the remainder instead of the number?
     9     76341    4213638.0     55.2     53.9          Digit = N % 10
    10     76341     188286.0      2.5      2.4          if Digit == 6:
    11      7925      21975.0      2.8      0.3              SixCounter   += 1
 

In [216]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 0.388465 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_16104\3149308276.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(N, D, Cache):
     4      1001       2521.0      2.5      0.1      SixCounter    = 0
     5      1001       2312.0      2.3      0.1      SixCounterMax = 0
     6      1001       2215.0      2.2      0.1      j = 0
     7     77342     194824.0      2.5      5.0      while N > 0:
     8     76341     157660.0      2.1      4.1          try:
     9     76341     265909.0      3.5      6.8              Q, R = Cache[j]
    10     76264     315681.0      4.1      8.1              R    = (R << 1) | (N & 1)
    11     76264     196525.0      2.6      5.1              if R >= D:
    12     37555     124209.0      3.3      3.2                  R = R - D
    13    

## Improve Candidate selection

So far we're checking every 399th digit of the number for a six, and if we find five such sixes in a row anywhere in the number, we declare the number a candidate for more thorough checking. This works very good and has given us a massive speedup already. However since the numbers are growing bigger and bigger, the chance of finding a candidate in this way also go up. At the start the probability was about 0.3%, but now it's become over 2%. This is not ideal since the thorough check is very expensive, and we want to do as little of them as possible.  
To reduce the likelyhood of detecting a candidate, we can simply check more digits. The more digits we check, the slower we progress however.  
What we've done here is to check more digits, but only if we've already detected 5 sixes in the usual way. On the fast pass this is only a single branch more whenever we hit a six, and on the slow pass, we now just start checking k digits between the 4th and 5th segment where we have already found the sixes at the start and end. All of these digits need to be sixes in order to be a goliath number, so a single non six immediately stops our search, and we condinue with taking large strides through the digits of the number.

In [149]:
def count_sixes_old(N, D, Cache):
    SixCounter    = 0
    SixCounterMax = 0
    j = 0
    while N > 0:
        try:
            Q, R = Cache[j]
            R    = (R << 1) | (N & 1)
            if R >= D:
                R = R - D
                N = (Q << 1) | 1
            else:
                N = (Q << 1)
        except KeyError:
            R = N  % D
            N = N // D
        Cache[j] = N, R
        j += 1
        
        Digit = R % 10
        if Digit == 6:
            SixCounter   += 1
            SixCounterMax = max(SixCounterMax, SixCounter)
        else:
            SixCounter = 0
        
    return SixCounterMax

In [151]:
profiler = LineProfiler()
@profiler
def count_sixes(N, D, Cache, k=1):
    SixCounter    = 0
    SixCounterMax = 0
    j = 0
    while N > 0:
        try:
            Q, R = Cache[j]
            R    = (R << 1) | (N & 1)
            if R >= D:
                R = R - D
                N = (Q << 1) | 1
            else:
                N = (Q << 1)
        except KeyError:
            R = N  % D
            N = N // D
        Cache[j] = N, R
        
        Digit = R % 10
        if Digit == 6:
            SixCounter += 1
            if SixCounter == 5:
                # Go back and run a more expensive check to reduce likelyhood of selecting (wrong) candidates.
                # If this is a goliath number, then this needs to now be all sixes.
                R = (Cache[j-1][1]) // 10 # We already know the last digit is a six, so skip it.
                for _ in range(k):
                    Digit = R % 10
                    if Digit == 6:
                        R = R // 10
                    else:
                        SixCounter = 1 # Set to one because the digit here at j is actually a 6.
                        break
            SixCounterMax = max(SixCounterMax, SixCounter) # This is not in the else block to support k = 0
        else:
            SixCounter = 0
        j += 1
    return SixCounterMax

In [54]:
%%time
i        = 100_000
Divisor = 10**399

Building Cache...
1
Done!
CPU times: total: 15.6 ms
Wall time: 17.1 ms


In [66]:
G = int('666'*666)

In [153]:
%%time
i      = 1_400_052
Number = 1<<(i)
count_sixes(Number, Divisor, {}, k=2)

CPU times: total: 3.3 s
Wall time: 3.3 s


4

In [154]:
count_sixes_old(Number, Divisor, {})

5

## Only store the last bit of each number, and the remainder?

Currently, for each step we take through the 399 size chunks of the number, we're storing the entire number left at that point, as well as the remainder. This is already a red flag, since if our cache is p segments long, that means that the very last segment (the very beginning of the number) is stored p times. The p-1th segment is stored p-1 times and so on, essentially creating a huge triangle if we were to visualise the data.  
When we look at the algorithm, we really only need the remainder and the last bit of the previous number. Even when we miss the cache, we have a pretty good idea of what the next number should be. Do some more testing, but a new segment added to the cache should always be a zero at first, and some remainder. 

In [423]:
def count_sixes_old(N, D, Cache, k=1):
    SixCounter    = 0
    SixCounterMax = 0
    j = 0
    while N > 0:
        try:
            Q, R = Cache[j]
            R    = (R << 1) | (N & 1)
            if R >= D:
                R = R - D
                N = (Q << 1) | 1
            else:
                N = (Q << 1)
        except KeyError:
            R = N  % D
            N = N // D
        Cache[j] = N, R
        
        Digit = R % 10
        if Digit == 6:
            SixCounter += 1
            if SixCounter == 5:
                R = (Cache[j-1][1]) // 10
                for _ in range(k):
                    Digit = R % 10
                    if Digit == 6:
                        R = R // 10
                    else:
                        SixCounter = 1
                        break
            SixCounterMax = max(SixCounterMax, SixCounter)
        else:
            SixCounter = 0
        j += 1
    return SixCounterMax

In [424]:
def RebuildCacheOld(LastNumber):
    """If the cache gets corrupted, use this to rebuild it.
    Be warned, this can take multiple seconds!"""
    Cache = {}
    i      = LastNumber
    Number = 1<<(i - 1)
    N = Number
    D = 10**399
    j = 0
    while N > 0:
        R = N  % D
        N = N // D
        Cache[j] = N, R
        j += 1
    return i, (Number<<1), D, Cache

In [425]:
def VerifyCacheOld(i, Number, Divisor, Cache):
    'If all of these checks pass, then it should be fine to use these values to continue the search.'
    assert Number  == 1<<i
    assert Divisor == 10**399
    assert (Number>>1) // Divisor == Cache[0][0]
    assert (Number>>1)  % Divisor == Cache[0][1]
    j            = len(Cache) - 1
    j0thDivision = (Number>>1) // (10**(399*(j  )))
    j1thDivision = (Number>>1) // (10**(399*(j+1)))
    assert j0thDivision != 0
    assert j0thDivision           == Cache[j-1][0]
    assert j1thDivision           == Cache[j][0] == 0
    assert j0thDivision % Divisor == Cache[j][1]
    assert j+1 not in Cache

In [428]:
def RebuildCache(LastNumber):
    """If the cache gets corrupted, use this to rebuild it.
    Be warned, this can take multiple seconds!"""
    i = LastNumber
    N = 1<<(i - 1)
    D = 10**399
    # j = 0
    Cache = []
    while N > 0:
        R = N  % D
        N = N // D
        Cache.append(R)
        # Cache[j] = R
        # j += 1
    return i, D, Cache

In [429]:
def VerifyCache(i, Divisor, Cache):
    'If all of these checks pass, then it should be fine to use these values to continue the search.'
    Number = 1<<(i - 1)
    assert Divisor == 10**399
    assert Number % Divisor == Cache[0]
    j            = len(Cache) - 1
    j0thDivision = Number // (10**(399*(j  )))
    j1thDivision = Number // (10**(399*(j+1)))
    assert j0thDivision != 0
    assert j1thDivision == 0
    assert j0thDivision % Divisor == Cache[j]
    assert Cache[-1] != 0

We iterate until N is no longer larger than 0, which means that at the last step, N will have to have been 0 for the loop to end.  
Let's say it is, then it can either be already in the cache, or it can be missing from the cache. In the case where we have a cache hit, we will increase R as usual and then check if it's become larger than D or not. If it hasn't, then N doesn't change. Shifting a zero to the left doesn't change the number, so it's still a zero, and the loop ends.  
If R has become larger than D, then we decrease R again, again shift the zero to the left, but now also "and" it with one. This means N is now 1, which means that the loop will continue.  
We will now enter the case where we have a cache miss.  
The new N and R are calculated by dividing the previous N, and taking the modulo of N respectively, each with the given Divisor. But we already know that the previous N must be a 1. Since D is larger than one, a floor division between 1 and D will always give 0 as the answer. Similarly, 1 % D will always give 1 as the answer.  
This means the new N will be 0, and the new R will be 1, no matter what happened in the iteration before.

In [437]:
# profiler = LineProfiler()
# @profiler
def count_sixes(D, Cache, k=1):
    SixCounter    = 0
    SixCounterMax = 0
    N = 0
    J = len(Cache)
    for j in range(J):
        R = (Cache[j] << 1) | N
        if R >= D:
            R = R - D
            N = 1
        else:
            N = 0
        Cache[j] = R
        
        Digit = R % 10
        if Digit == 6:
            SixCounter += 1
            if SixCounter == 5:
                R = (Cache[j-1]) // 10
                for _ in range(k):
                    Digit = R % 10
                    if Digit == 6:
                        R = R // 10
                    else:
                        SixCounter = 1
                        break
            SixCounterMax = max(SixCounterMax, SixCounter)
        else:
            SixCounter = 0
    if N > 0:
        # print('hi')
        # Cache[J] = 1
        Cache.append(1)
    return SixCounterMax

In [447]:
%%time
i, Number, Divisor, Cache = RebuildCacheOld(1_000_000)
VerifyCacheOld(i, Number, Divisor, Cache)

CPU times: total: 1.64 s
Wall time: 1.64 s


In [448]:
%%time
output = []
for _ in range(1000):
    output.append(count_sixes_old(Number, Divisor, Cache))
    i      += 1
    Number  = Number<<1

CPU times: total: 9.94 s
Wall time: 9.96 s


In [ ]:
# New

In [449]:
%%time
i, Divisor, Cache = RebuildCache(1_000_000)
VerifyCache(i, Divisor, Cache)

CPU times: total: 1.62 s
Wall time: 1.64 s


In [450]:
%%time
output2 = []
for _ in range(1_000):
    output2.append(count_sixes(Divisor, Cache))
    i      += 1

CPU times: total: 297 ms
Wall time: 305 ms


In [451]:
output == output2

True

In [455]:
VerifyCache(i, Divisor, Cache)

Crushing defeat!!!  

The new version is at least 5x faster than the old one.

In [418]:
profiler.print_stats()

Timer unit: 1e-07 s

Total time: 7.88987 s
File: C:\Users\Florian\AppData\Local\Temp\ipykernel_20112\3990742229.py
Function: count_sixes at line 2

Line #      Hits         Time  Per Hit   % Time  Line Contents
     2                                           @profiler
     3                                           def count_sixes(D, Cache, k=1):
     4      1000       4742.0      4.7      0.0      SixCounter    = 0
     5      1000       3561.0      3.6      0.0      SixCounterMax = 0
     6      1000       2571.0      2.6      0.0      N = 0
     7      1000       8541.0      8.5      0.0      J = len(Cache)
     8   3019792    7982991.0      2.6     10.1      for j in range(J):
     9   3018792   12152451.0      4.0     15.4          R = (Cache[j] << 1) | N
    10   3018792    7493703.0      2.5      9.5          if R >= D:
    11   1508042    4779038.0      3.2      6.1              R = R - D
    12   1508042    3723418.0      2.5      4.7              N = 1
    13               

## Implementation Optimization

In [4]:
Cache = [10**399] * 4000

In [14]:
%%timeit
J = len(Cache)
for j in range(J):
    R = Cache[j]
    # Do stuff with R
    Cache[j] = R

217 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [15]:
%%timeit
for j in range(len(Cache)):
    R = Cache[j]
    # Do stuff with R
    Cache[j] = R

216 µs ± 913 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [16]:
%%timeit
for (j, R) in enumerate(Cache):
    # Do stuff with R
    Cache[j] = R

181 µs ± 646 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
